In [ ]:
import random
import requests
import pandas as pd
from datasets import load_dataset
import json

In [ ]:
def extract_annotation_text(description_dom):
    """
    Извлекает текст из DOM-структуры описания песни с добавлением пробелов между частями текста.
    :param description_dom: DOM-структура, содержащая описание.
    :return: Строка с полным текстом описания.
    """
    def parse_dom(dom):
        """
        Рекурсивно обходит DOM-структуру для извлечения текста.
        :param dom: Элемент DOM-структуры.
        :return: Текст из элемента DOM.
        """
        if isinstance(dom, str):
            return dom.strip()
        if isinstance(dom, dict):
            if "children" in dom:
                return " ".join([parse_dom(child) for child in dom["children"]])
            return ""
        if isinstance(dom, list):
            return " ".join([parse_dom(child) for child in dom])
        return ""

    return " ".join(parse_dom(description_dom).split())


In [ ]:
# Токен для доступа к Genius API (замените на свой токен)
GENIUS_API_TOKEN = "TOKEN"

# Функция для получения информации о песне
def get_song_info(song_title, artist_name=None):
    base_url = "https://api.genius.com/search"
    headers = {
        "Authorization": f"Bearer {GENIUS_API_TOKEN}"
    }
    search_input = song_title + " " + artist_name
    print(search_input)
    params = {
        "q": search_input
    }

    try:
        # Выполнение поиска по названию песни
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        best_variants = search_results["response"]["hits"]
        song_data = None
        for e in best_variants:
          print(e)
          if artist_name in e["result"]["artist_names"].replace("\xa0", " "):
            song_data = e['result']
            break
        if song_data is None:
          return None

        # Получение полной информации о песне
        song_id = song_data["id"]
        song_url = f"https://api.genius.com/songs/{song_id}"
        song_response = requests.get(song_url, headers=headers)
        song_response.raise_for_status()
        song_info = song_response.json()["response"]["song"]
        print(song_info)

        # Возвращаем данные о песне
        return song_info

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса к API Genius: {e}")
        return None

# Пример использования функции
if __name__ == "__main__":
    song_title = "Фанк"
    artist_name = "Валентин Стрыкало"
    song_info = get_song_info(song_title, artist_name)

    if song_info:
        print("Название:", song_info.get("title"))
        print("Артист:", song_info.get("artist_names"))
        print("Альбом:", song_info.get("album", {}).get("name"))
        print("Дата выпуска:", song_info.get("release_date_for_display"))
        print("Описание:", extract_annotation_text(song_info.get("description", {}).get("dom", {}).get("children")))
        print("Ссылка на песню:", song_info.get("url"))


Фанк Валентин Стрыкало
{'highlights': [], 'index': 'song', 'type': 'song', 'result': {'annotation_count': 2, 'api_path': '/songs/3944499', 'artist_names': 'Валентин Стрыкало (Valentin Strikalo)', 'full_title': 'Фанк (Funk) by\xa0Валентин\xa0Стрыкало (Valentin Strikalo)', 'header_image_thumbnail_url': 'https://images.genius.com/f5c062b72d4d8abaae472b787d1b133a.300x300x1.png', 'header_image_url': 'https://images.genius.com/f5c062b72d4d8abaae472b787d1b133a.1000x1000x1.png', 'id': 3944499, 'lyrics_owner_id': 4948196, 'lyrics_state': 'complete', 'path': '/Valentin-strikalo-funk-lyrics', 'primary_artist_names': 'Валентин Стрыкало (Valentin Strikalo)', 'pyongs_count': 4, 'relationships_index_url': 'https://genius.com/Valentin-strikalo-funk-sample', 'release_date_components': {'year': 2012, 'month': 2, 'day': 24}, 'release_date_for_display': 'February 24, 2012', 'release_date_with_abbreviated_month_for_display': 'Feb. 24, 2012', 'song_art_image_thumbnail_url': 'https://images.genius.com/f5c062

In [ ]:
# Токен для доступа к Genius API (замените на свой токен)
GENIUS_API_TOKEN = "TOKEN"

# Функция для получения информации о песне
def get_song_info(song_title, artist_name=None):
    base_url = "https://api.genius.com/search"
    headers = {
        "Authorization": f"Bearer {GENIUS_API_TOKEN}"
    }
    params = {
        "q": song_title
    }

    try:
        # Выполнение поиска по названию песни
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        best_variants = search_results["response"]["hits"]
        for e in best_variants:
          print()
          if e["result"]["artist_names"] == artist_name:
            song_data = e['result']
            break


        # Получение полной информации о песне
        song_id = song_data["id"]
        song_url = f"https://api.genius.com/songs/{song_id}"
        song_response = requests.get(song_url, headers=headers)
        song_response.raise_for_status()
        song_info = song_response.json()["response"]["song"]

        # Возвращаем данные о песне
        return song_info

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса к API Genius: {e}")
        return None



In [ ]:
if __name__ == "__main__":
    song_title = "TOXIC"
    artist_name = "NTNka"
    song_info = get_song_info(song_title, artist_name)

    if song_info:
        print("Название:", song_info.get("title"))
        print("Артист:", song_info.get("primary_artist", {}).get("name"))
        print("Альбом:", song_info.get("album", {}).get("name"))
        print("Дата выпуска:", song_info.get("release_date_for_display"))
        print("Описание:", extract_annotation_text(song_info.get("description", {}).get("dom", {}).get("children")))
        print("Ссылка на песню:", song_info.get("url"))


UnboundLocalError: local variable 'song_data' referenced before assignment

In [ ]:
ds = load_dataset("sevenreasons/genius-lyrics-russian", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
n = 10
start = 1
end =   192020

unique_random_integers = [311]
print(unique_random_integers)

df = pd.DataFrame(ds.select(unique_random_integers))
df[["title", "artist"]]

[311]


,title,artist
0,Месяц май (Mesyats may),Юля Паршута (Parshoota)


In [ ]:
song_title = "Всё как у людей (Everything is Like People’s)"
artist_name = "Noize MC"
song_info = get_song_info(song_title, artist_name)

if song_info:
    print("Название:", song_info.get("title"))
    print("Артист:", song_info.get("primary_artist", {}).get("name"))
    print("Описание:", extract_annotation_text(song_info.get("description", {}).get("dom", {}).get("children")))


{'annotation_count': 11, 'api_path': '/songs/4856014', 'apple_music_id': None, 'apple_music_player_url': 'https://genius.com/songs/4856014/apple_music_player', 'artist_names': 'Noize\xa0MC', 'description': {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['В честь дня рождения Егора Летова Noize MC представил трек, который вошёл в сборник под названием ', {'tag': 'em', 'children': [{'tag': 'a', 'attributes': {'href': 'https://genius.com/albums/Various-artists/Without-me-tribute-to-eegor-letov', 'rel': 'noopener'}, 'data': {'api_path': '/albums/561191'}, 'children': ['Без меня. Трибьют Егора Летова']}]}, '.']}, '', {'tag': 'p', 'children': ['Огромное количество известных и не очень артистов перепели песни великого музыканта и поэта, однако Noize MC решил лишь засемплировать припев оригинальной песни и дописать к ней четыре куплета. Трек «Всё как у людей» выпущен как бонус для сборника.']}, '', {'tag': 'p', 'children': ['Noize MC также представил ', {'tag': 'a', 'attributes'

In [ ]:
# Функция для извлечения аннотации
def fetch_annotation(song_title, artist_name=None):
    """
    Получает аннотацию для песни с использованием Genius API.
    :param song_title: Название песни.
    :param artist_name: Имя артиста.
    :return: Текст аннотации или None.
    """
    try:
        song_info = get_song_info(song_title, artist_name)
        if not song_info:
            return None

        # Получаем описание в формате DOM
        description_dom = song_info.get("description", {}).get("dom", {}).get("children", "?")
        if description_dom == "?":
            return None

        # Извлекаем текст аннотации
        annotation_text = extract_annotation_text(description_dom)
        return annotation_text
    except Exception as e:
        print(f"Ошибка при обработке песни '{song_title}': {e}")
        return None

# Функция для добавления столбца annotation
def add_annotation_column(df):
    """
    Добавляет столбец `annotation` в DataFrame с аннотациями песен.
    :param df: DataFrame с колонками `song_title` и `artist_name`.
    :return: DataFrame с добавленным столбцом `annotation`.
    """
    annotations = []

    for index, row in df.iterrows():
        song_title = row.get("title")
        artist_name = row.get("artist")


        print(f"Обработка песни: {song_title} ({artist_name})")
        annotation = fetch_annotation(song_title, artist_name)
        annotations.append(annotation)

    # Добавляем новый столбец
    df["annotation"] = annotations
    return df

# Загрузка набора данных и добавление аннотаций
if __name__ == "__main__":
    ds = load_dataset("sevenreasons/genius-lyrics-russian", split="train")
    df = pd.DataFrame(ds.select(unique_random_integers))

    # Добавляем столбец с аннотациями
    updated_df = add_annotation_column(df)

    # Сохраняем обновленный DataFrame
    updated_df.to_csv("genius_lyrics_with_annotations.csv", index=False)
    print("Обновленный DataFrame сохранен в файл genius_lyrics_with_annotations.csv")


Обработка песни: D’Lux Promo (Теона Дольникова (Teona Dolnikova))
{'annotation_count': 0, 'api_path': '/songs/7603268', 'apple_music_id': None, 'apple_music_player_url': 'https://genius.com/songs/7603268/apple_music_player', 'artist_names': 'Dino\xa0MC47 (Ft. N’Pans & Теона Дольникова (Teona Dolnikova))', 'description': {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['?']}]}}, 'embed_content': "<div id='rg_embed_link_7603268' class='rg_embed_link' data-song-id='7603268'>Read <a href='https://genius.com/Dino-mc47-dlux-promo-lyrics'>“D’LUX Promo” by Dino\xa0MC47</a> on Genius</div> <script crossorigin src='//genius.com/songs/7603268/embed.js'></script>", 'featured_video': False, 'full_title': 'D’LUX Promo by\xa0Dino\xa0MC47 (Ft.\xa0N’Pans & Теона\xa0Дольникова (Teona Dolnikova))', 'header_image_thumbnail_url': 'https://images.genius.com/e6516c5f0ad08ef12ef05767aa2b5182.300x300x1.jpg', 'header_image_url': 'https://images.genius.com/e6516c5f0ad08ef12ef05767aa2b5182.496x496x1

In [ ]:
# Загрузка JSON файла
with open("/content/part_10.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Подсчет общего количества элементов в JSON файле
total_elements = len(data)

# Подсчет элементов, где значение 'annotation' не является null или "?"
valid_annotations = sum(1 for item in data if item.get("annotation") not in (None, "?"))

# Вывод результатов
print(f"Общее количество элементов: {total_elements}")
print(f"Количество элементов с валидными 'annotation': {valid_annotations}")


Общее количество элементов: 5062
Количество элементов с валидными 'annotation': 508
